# autoML with auto-keras

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

## prep the data for auto-keras

In [2]:
diabetes = datasets.load_diabetes()
dia_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
dia_df.columns = ['age',
  'sex',
  'bmi',
  'bp',
  'tc',
  'ldl',
  'hdl',
  'tch',
  'ltg',
  'glu']
dia_df["target"] = diabetes.target
df_train, df_test = train_test_split(dia_df, train_size=0.8, test_size=0.2, random_state=1231)
df_train.to_csv("diabetes_train.csv", index=False)
df_test.to_csv("diabetes_test.csv", index=False)

In [24]:
# max_trials, the number of different models to try
autokeras_reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=4
)  

train_data_path="diabetes_train.csv"
autokeras_reg.fit(
    # The path to the train.csv file.
    train_data_path,
    # The name of the outcome column.
    "target",
    epochs=100,
)

Trial 4 Complete [00h 00m 07s]
val_loss: 3475.74267578125

Best val_loss So Far: 2945.620361328125
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
12/12 [==============================] - 0s 2ms/step - loss: 28712.7344 - mean_squared_error: 28712.7344
Epoch 2/100
12/12 [==============================] - 0s 1ms/step - loss: 27504.2383 - mean_squared_error: 27504.2383
Epoch 3/100
12/12 [==============================] - 0s 1ms/step - loss: 25398.4023 - mean_squared_error: 25398.4023
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 22052.7539 - mean_squared_error: 22052.7539
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 17480.2715 - mean_squared_error: 17480.2715
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 12363.0117 - mean_squared_error: 12363.0117
Epoch 7/100
12/12 [==============================] - 0s 1ms/step - loss: 8206.6230 - mean_squared_error: 8206.6230
Epoch 8/100
12

## generate the prediction based on the aukeras model

In [25]:
y_test_predicted_ak=autokeras_reg.predict(df_test.iloc[:,0:10])

3/3 [==============================] - 0s 2ms/step


## Performance metrics using r2 and mean absolute error

Change the epoch hyperparameter to see what is optimal choice

In [26]:
from sklearn.metrics import r2_score
r2_score(df_test["target"],y_test_predicted_ak)

0.54979672276334

summary of r2_score:
- epoch of 10: -0.46259136836515946
- epoch of 50: 0.3834656224342138
- epoch of 100: 0.54979672276334
- epoch of 150: 0.5094809564552147
- epoch of 1000: 0.49017001009308037

In [27]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_test["target"],y_test_predicted_ak)

42.490659263696564

summary of mean_absolute_error
epoch of 10: 73.18183654613709
epoch of 50: 50.35545091950492
epoch of 100: 42.490659263696564
epoch of 150: 43.45209288864993
epoch of 1000: 44.13179123267699

## Conclusion

It is clear that auto-keras is very powerful and easy-to-use.

The example we have here using structured data indicates for the autoML workflow there is a need for fine-tuning of the epoch hyperparameter to avoid under-fitting (when number of epoch is low, e.g, 10 or 50), or over-fitting (with number of epochs of 2150 or 1000).

The best result is with epoch of 100, with performance similar to (and slightly worse than) the tpot results.

Due to the tuning of hyperparameters, we can say the auto-keras library is in a way not so "auto", compared to tpot and hyperopt-sklearn. However, it does provide the tuning and search capability, and it works with more different kinds of models.